In [ ]:
#goal: have a table that looks like this
#how to 

'''
|country|first_idx|second_idx|third_idx|

switzerland 1       13         12



////

then incorporate weights



tables DONE:
* salary (242) (biggest list - treated as master)
* safety (163); 92 404s? need to inspect
* life expectancy (183)
* days off (194)

to-do:


------Q------

need to do manual inspection for some countries (e.g. safety - Luxembourg)



----Notes----
NaN replaced with 404







In [1]:
import pandas as pd

sal = pd.read_csv("salary.csv", delimiter = ';', header = None, index_col = False)
sal.rename(columns = {0: 'salary idx', 1: 'Country', 2: 'Salary', 3: 'Currency', 4: 'Salary[USD]'}, inplace = True)
sal.sort_values(by = 'Salary[USD]', ascending = False, inplace = True)
sal.reset_index(drop=True, inplace=True)  
sal.tail()
sal.dropna(inplace = True)

countries = []
for country in sal['Country']:
    countries.append(country)

master = pd.DataFrame(columns = ["Country", "Salary", "Safety", "Life Expectancy", "Days Off"])
master.head()
master["Country"] = countries

master.head()



#sal.drop(['NaN'])

,Country,Salary,Safety,Life Expectancy,Days Off
0,Switzerland,NaN,NaN,NaN,NaN
1,United States,NaN,NaN,NaN,NaN
2,Denmark,NaN,NaN,NaN,NaN
3,Japan,NaN,NaN,NaN,NaN
4,Luxembourg,NaN,NaN,NaN,NaN


In [4]:
def master_add(column, listname = []):
    
    #premise: we're not adding the values, but the position in the ranking    
    
    for country in countries:
        try:
            value = listname[listname['Country']==country].index.item()
            real_val = listname.at[value, column] # 
            
      #      value = listname[listname['Country']==country].        
    #   print country, value, real_val
            value+=1
        except: #Country not found
            #print "error"
            value = 404 #bc not found
        
        m_val = master[master['Country'] == country].index.item()   #not in the try block bc master has all the countries
#        master.at[m_val, column] = value #doesn't work?
        master.at[m_val, column] = value # real_val
        
        
#    print m_val, column, value
    
    
    
  #  return master
    

In [5]:
master_add("Salary", sal)

In [6]:
print master.head()

         Country Salary Safety Life Expectancy Days Off
0    Switzerland      1    NaN             NaN      NaN
1  United States      2    NaN             NaN      NaN
2        Denmark      3    NaN             NaN      NaN
3          Japan      4    NaN             NaN      NaN
4     Luxembourg      5    NaN             NaN      NaN


In [8]:
#already sorted

saf = pd.read_csv("scrapers/safety.csv", delimiter = ';', header = None, index_col = False)
saf.rename(columns = {0: 'Safety', 1: 'Country', 2: 'Score'}, inplace = True)
saf.drop(columns = "Score", axis = 1, inplace = True)

saf.head()
#saf.head()

,Safety,Country
0,1,Iceland
1,2,New Zealand
2,3,Portugal
3,4,Austria
4,5,Denmark


In [9]:
master_add("Safety", saf)

In [11]:
#already sorted

life_exp = pd.read_csv("scrapers/life-exp.csv", delimiter = ';', header = None, index_col = False)
life_exp.rename(columns = {0: 'Life Expectancy', 1: 'Country', 2: 'Predicted Score'}, inplace = True)
life_exp.drop(columns = "Predicted Score", inplace = True)
life_exp.head()



,Life Expectancy,Country
0,1,Japan
1,2,Switzerland
2,3,Singapore
3,4,Australia
4,5,Spain


In [12]:
master_add("Life Expectancy", life_exp)

In [13]:
master.head()

,Country,Salary,Safety,Life Expectancy,Days Off
0,Switzerland,1,10,2,NaN
1,United States,2,128,31,NaN
2,Denmark,3,5,27,NaN
3,Japan,4,9,1,NaN
4,Luxembourg,5,404,13,NaN


In [14]:
days_off = pd.read_csv("scrapers/days_off.csv", delimiter = ';', header = None, index_col = False)
days_off.sort_values(by = 2, ascending = False, inplace = True)
days_off.reset_index(drop=True, inplace=True)  
days_off.drop(columns = 0, inplace = True)
days_off.rename(columns = {1: 'Country', 2: 'Days Off'}, inplace = True)



days_off.head()

len(days_off)

194

In [15]:
master_add("Days Off", days_off)

In [16]:
master.head()

,Country,Salary,Safety,Life Expectancy,Days Off
0,Switzerland,1,10,2,95
1,United States,2,128,31,188
2,Denmark,3,5,27,23
3,Japan,4,9,1,185
4,Luxembourg,5,404,13,7


In [40]:
def point_add(category):
    new_col = category + " points"
    master[new_col] = ""
    for country in countries:
        value = master[master['Country']==country]
        place = value[category]
        print country, place

        if place < 10:
                x = 10
        else:
                x = 100

        value[category] = x


    

In [41]:
point_add("Safety")

Switzerland 0    10
Name: Safety, dtype: object


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [22]:
master.head()

,Country,Salary,Safety,Life Expectancy,Days Off,test,testpoints
0,Switzerland,1,10,2,95,,
1,United States,2,128,31,188,,
2,Denmark,3,5,27,23,,
3,Japan,4,9,1,185,,
4,Luxembourg,5,404,13,7,,


In [11]:
days_off.iloc[[50]]
days_off[days_off['Country']=="United States"].index.item() 
try:
    val = days_off[days_off['Country']=="United Kingdom"].index.item()
    print val
except: # country not in the list
    print "na"
        


83


In [29]:
lan = pd.read_csv("scrapers/languages.csv", delimiter = ';', header = None, index_col = False)

'''
days_off.sort_values(by = 2, ascending = False, inplace = True)
days_off.reset_index(drop=True, inplace=True)  
days_off.drop(columns = 0, inplace = True)
days_off.rename(columns = {1: 'Country', 2: 'Days Off'}, inplace = True)

'''

lan.head()


#this is going to be used to determine whether the country speaks your language
# move to a later phase


,0,1,2
0,1,Albania,"Albanian (Tosk is the official dialect), Greek"
1,2,Algeria,"Arabic (official), French, Berber dialects"
2,3,Andorra,"Catalán (official), French, Castilian, Portuguese"
3,4,Angola,"Portuguese (official), Bantu and other African..."
4,5,Antigua and Barbuda,"English (official), local dialects"


In [51]:
taxes = pd.read_csv("scrapers/taxes.csv", delimiter = ';', header = None, index_col = False)
taxes.sort_values(by = 2, ascending = True, inplace = True)
taxes.drop(columns = 0, inplace = True)
taxes.reset_index(drop = True, inplace = True)
taxes.rename(columns = {1: "Country", 2: "Income Tax"}, inplace = True)
taxes.head(10)

,Country,Income Tax
0,United Arab Emirates,0.0
1,Bahamas,0.0
2,Bahrain,0.0
3,Bermuda,0.0
4,Brunei,0.0
5,Saudi Arabia,0.0
6,Kuwait,0.0
7,Oman,0.0
8,Qatar,0.0
9,Cayman Islands,0.0


In [52]:
master_add("Income Tax", taxes)
master.head()

Income Tax
115.0


,Country,Salary,Safety,Life Expectancy,Days Off,Income Tax
0,Switzerland,1,404,NaN,NaN,123.0
1,United States,2,404,NaN,NaN,115.0
2,Denmark,3,404,NaN,NaN,152.0
3,Japan,4,404,NaN,NaN,153.0
4,Luxembourg,5,404,NaN,NaN,144.0


In [28]:
master.head(20)
#wrong value for Denmark - idk why
# and wrong value for the US in days off

# look into the master_add - doesn't seem to be working correctly


,Country,Salary,Safety,Life Expectancy,Days Off,test,testpoints,test points
0,Switzerland,1,10,2,95,,,
1,United States,2,128,31,188,,,
2,Denmark,3,5,27,23,,,
3,Japan,4,9,1,185,,,
4,Luxembourg,5,404,13,7,,,
5,Singapore,6,7,3,159,,,
6,Norway,7,20,15,29,,,
7,Hong Kong,8,404,404,153,,,
8,United Kingdom,9,45,20,84,,,
9,New Zealand,10,2,17,67,,,


In [ ]:
'''
ideas for distribution:
1. points
    100 points in total
    by choosing which category is more important -> that category gets more points
    then in each category can assign the points by choice:
            eg. if max 10 points to distribute
                    1-10: 10
                    10-20: 8
                    
                    .. etc (Nan's get 0 - but need to make sure which ones are actually NaN - manual inspection)
                    
                    if some category worth X points:
                    1 - 10: x
                    10 - 20: 0.9x
                    20 - 30: 0.8x
                    30 - 40: 0.7x
                    40 - 50: 0.6x
                    50 - 100: 0.5x
                    100 - 150: 0.3x
                    150+ : 0?
                    
            but it should be different dependent on the category
            e.g. the differences in the income tax are not significant between 110th and 150th i think
                    

next phases of work:
    * FIRST - fix the table
    * reading the remaining csv's
    OR * experimenting with the basic functionality    
    btw remember that salary is the only column that changes - the rest can just be generated and re-loaded




'''

In [15]:
#countring the ones that are 404
saf_null = master[master["Safety"] == 404].count()
life_exp_null = master[master["Life Expectancy"] == 404].count()
days_off_null = master[master["Days Off"] == 404].count()

print saf_null, life_exp_null, days_off_null


Country            92
Salary             92
Safety             92
Life Expectancy    92
Days Off           92
dtype: int64 Country            68
Salary             68
Safety             68
Life Expectancy    68
Days Off           68
dtype: int64 Country            61
Salary             61
Safety             61
Life Expectancy    61
Days Off           61
dtype: int64
